In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import Sequence, to_categorical
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger
from tensorflow.keras.metrics import AUC

from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import StandardScaler

2025-05-15 17:03:53.944274: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from tensorflow.keras.models import load_model

# Build and Train CNN

In [2]:
def train(X,y,groups):

    # init for tracking
    all_models = []
    all_histories = []

    gkf = GroupKFold(n_splits=5, shuffle=True)    

    # using trainign data, holdout set reserved
    for ifold, (train_idx, val_idx) in enumerate(gkf.split(X, y, groups=groups)):
        print(f"\nRunning fold {ifold+1}/5")
        # get train and test data
        Xtr, Xval = X[train_idx], X[val_idx]
        ytr, yval = y[train_idx], y[val_idx]

        # Flatten time for scasling
        Xtr_reshaped = Xtr.reshape(-1, Xtr.shape[2]) # remove num_windows dimension for scaling
        Xval_reshaped = Xval.reshape(-1, Xval.shape[2])

        # scale
        scaler = StandardScaler()
        Xtr_scal = scaler.fit_transform(Xtr_reshaped).reshape(Xtr.shape) # restore original shape
        Xval_scal = scaler.transform(Xval_reshaped).reshape(Xval.shape)

        # define checkpoints
        # number of epochs with no improvment after which the training will stop
        early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, verbose=2)

        # save best model
        checkpoint = ModelCheckpoint(
        "model-epoch{epoch:02d}-valacc{val_accuracy:.2f}.keras", # save as .keras next time
        monitor="val_accuracy",
        save_best_only=True,
        verbose=2
    )

        # Build model  # input shape means a sinlge SAMple is of shape 200 rows * num features
        model = build_model(input_shape=(X.shape[1], X.shape[2]), num_classes=1)

        # compile
        model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', AUC(name='auc')])

        csv_logger = CSVLogger('training_log.csv', append=False) # to access metrics per epoch
        
        history = model.fit(
            Xtr_scal, ytr,
            validation_data=(Xval_scal, yval),
            epochs=20,
            batch_size=32, # 32 windows in 1 batch, with every batch per epoch, updates weights
            verbose=2,
            callbacks = [checkpoint, early_stopping, csv_logger]
        )

    all_models.append(model)
    all_histories.append(history)

    return all_models, all_histories


In [3]:
def build_model(input_shape=(500, 144), num_classes=1): # input: 144-length vectors with 500 timesteps

    model = models.Sequential([
        # First block: quarter-second time steps
        # kernel size = temporal width of the filter #filter size = num conv filters learned
        layers.Input(shape=input_shape),
        layers.Conv1D(32, kernel_size=25, padding='same', activation='relu'), #filter size = num conv filters learned
        layers.BatchNormalization(),
        layers.Dropout(0.3),

        # 1-second time steps
        layers.Conv1D(64, kernel_size=50, dilation_rate=2, padding='same', activation='relu'), # padding = same, output shape remains 500. (500, 64)
        layers.BatchNormalization(),
        layers.MaxPooling1D(pool_size=2), # downsamples by taking max activation(strongest patterns) 

        # 5-second time steps
        layers.Conv1D(128, kernel_size=50, dilation_rate=10, padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling1D(pool_size=2), 

        # summarize
        layers.GlobalAveragePooling1D(), # flattens
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.5),  # randomly zeroes out 32 of the 64 activations each epoch
        layers.Dense(num_classes, activation='sigmoid')  # binary classification
    ])

    return model


In [5]:
if __name__ == "__main__":
    # load data: 80% of preprocessed data with 20% saved for a holdout
    y = np.load("/home/sbinder3/wearables/y_train.npy")
    X = np.load("/home/sbinder3/wearables/X_train.npy")
    groups = np.load("/home/sbinder3/wearables/groups_train.npy")

    
    # call function
    models, histories = train(X,y,groups)


Epoch 1/20



Epoch 1: val_accuracy improved from -inf to 0.90202, saving model to model-epoch01-valacc0.90.keras


90/90 - 32s - 351ms/step - accuracy: 0.8082 - auc: 0.8811 - loss: 0.4725 - val_accuracy: 0.9020 - val_auc: 0.9657 - val_loss: 0.2671


Epoch 2/20



Epoch 2: val_accuracy improved from 0.90202 to 0.91066, saving model to model-epoch02-valacc0.91.keras


...


Epoch 19: val_accuracy did not improve from 0.96387


90/90 - 29s - 324ms/step - accuracy: 0.9913 - auc: 0.9996 - loss: 0.0240 - val_accuracy: 0.9538 - val_auc: 0.9775 - val_loss: 0.2309


Epoch 20/20



Epoch 20: val_accuracy did not improve from 0.96387


90/90 - 29s - 323ms/step - accuracy: 0.9930 - auc: 0.9995 - loss: 0.0188 - val_accuracy: 0.9465 - val_auc: 0.9736 - val_loss: 0.2469


# Evaluate


In [30]:
# load best model
model = load_model("model-epoch09-valacc0.97.keras")

# compile
model.compile(optimizer='adam',
      loss='binary_crossentropy',
      metrics=['accuracy', AUC(name='auc')])

results = model.evaluate(X_test, y_test, verbose=2)
print(f"Test Accuracy: {results[1]:.4f}")
print(f"Test AUC: {results[2]:.4f}")


29/29 - 2s - 79ms/step - accuracy: 0.7364 - auc: 0.7384 - loss: 46.8866
Test Accuracy: 0.7364
Test AUC: 0.7384


In [33]:
#model.summary()

In [19]:
# load holdout set
y_test = np.load("/home/sbinder3/wearables/y_test.npy")
X_test = np.load("/home/sbinder3/wearables/X_test.npy")

# Prev versions

In [9]:
# X = np.array(all_X)
# y = np.array(all_y)
# groups = np.array(all_groups)

# def train(X,y,groups):

#     # init for tracking
#     all_models = []
#     all_histories = []

#     gkf = GroupKFold(n_splits=5, shuffle=True)     # data shuffle?
    
#     for ifold, (train_idx, val_idx) in enumerate(gkf.split(X, y, groups=groups)):
#         print(f"\nRunning fold {ifold_+1}/5")
#         # get train and test data
#         Xtr, Xval = X[train_idx], X[val_idx]
#         ytr, yval = y[train_idx], y[val_idx]

#         #scale
#         scaler = StandardScaler()
#         Xtr_scal = scaler.fit_transform(Xtr)
#         Xval_scal = scaler.transform(Xval)

#         # define checkpoints
#         # number of epochs with no improvment after which the training will stop
#         early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, verbose=1)
        
#         # save best model
#         checkpoint = ModelCheckpoint(
#         "model-epoch{epoch:02d}-valacc{val_accuracy:.2f}.h5",
#         monitor="val_accuracy",
#         save_best_only=True,
#         verbose=2
#     )

#         # Build model  # input shape means a sinlge SAMple is of shape 200 rows * num features
#         model = build_model(input_shape=(X.shape[1], X.shape[2]), num_classes=3) 

#         # compile
#         model.compile(optimizer='adam',
#               loss='categorical_crossentropy',
#               metrics=['accuracy'])
    
#         model.fit(
#             Xtr_scal, ytr
#             validation_data=(Xval_scal, yval),
#             epochs=20,
#             batch_size=32,
#             verbose=2,
#             callbacks = [checkpoint, early_stopping]
#         )

#     all_models.append(model)
#     all_histories.append(history)

#     return all_models, all_histories


In [10]:
# def build_model(input_shape=(200, 162), num_classes=3): # input: 162-length vectors with 200 timesteps # BATCH size?
    
#     model = models.Sequential([
#         # First block: half-second time steps
#         # kernel size = temporal width of the filter #filter size = num conv filters learned
#         layers.Conv1D(64, kernel_size=25, dilation_rate=2, padding='same', activation='relu', input_shape=input_shape)), #filter size = num conv filters learned 
#         layers.BatchNormalization(),
#         layers.Dropout(0.3),

#         # 1-second time steps
#         layers.Conv1D(128, kernel_size=50, dilation_rate=2, padding='same', activation='relu') # padding = same, output shape remains 200. (200, 128)
#         layers.BatchNormalization(),
#         layers.MaxPooling1D(pool_size=2), # downsamples by taking max activation(strongest patterns) # 200 > 100

#         # 2-second time steps
#         layers.Conv1D(128, kernel_size=50, dilation_rate=4, padding='same', activation='relu') 
#         layers.BatchNormalization(),
#         layers.MaxPooling1D(pool_size=2), # 100 > 50

#         # summarize
#         layers.GlobalAveragePooling1D(), # flattens
#         layers.Dense(64, activation='relu'),
#         layers.Dropout(0.5),  # randomly zeroes out 32 of the 64 activations each epoch
#         layers.Dense(num_classes, activation='softmax')  
#     ])
    
#     return model
